# 一、Basic

## 1.1 大数定理

导入必要包

In [ ]:
%matplotlib inline
import torch
from torch.distributions import multinomial # 从概率分布包中导入多项分布
from d2l import torch as d2l

多项分布(multionmial distribution)：将概率分配给一些离散选择的分布

下模拟掷色子：

In [ ]:
fair_probs = torch.ones(6) / 6 # 提供概率分布张量
multinomial.Multinomial(1, fair_probs).sample() # sample()方法根据提供的概率和总模拟次数total_count产生一个可能的样本
# 可以将total_count参数赋其它的值n，以模拟n次投掷
multinomial.Multinomial(1000, fair_probs).sample() / 1000 # 模拟投掷1000次的概率
fair_probs - multinomial.Multinomial(1000, fair_probs).sample() / 1000 # 计算概率误差

`sample()`方法也可以接受一个可迭代的`sample_shape`参数，表示产生的样本总数

In [ ]:
multinomial.Multinomial(1, fair_probs).sample((100,)) # 共100组模拟掷色子实验，每组扔1次，返回一个规模为100*6的张量（6为概率张量的大小）
multinomial.Multinomial(1, fair_probs).sample((100, 100)) # 对上一排中的实验重复进行100次

进行多组模拟并绘图

In [ ]:
counts = multinomial.Multinomial(100, fair_probs).sample((500,)) # 进行500组模拟，每组实验100次
cum_counts = counts.cumsum(dim=0) # 累次相加，计算所有组中获得同一结果的总数
estimates = cum_counts / cum_counts.sum(dim=1, keepdim=True) # 计算模拟概率。keepdim参数使返回的张量（原本将会降维）保持维数

d2l.set_figsize((6, 4.5))
for i in range(6):
    d2l.plt.plot(estimates[:, i].numpy(),label=("P(die=" + str(i + 1) + ")"))
d2l.plt.axhline(y=0.167, color='black', linestyle='dashed') 
d2l.plt.gca().set_xlabel('Groups of experiments')
d2l.plt.gca().set_ylabel('Estimated probability')
d2l.plt.legend()

# 二、定理

1. **贝叶斯定理**：
$$
P(A \mid B) = \frac{P(B \mid A) P(A)}{P(B)}.
$$
来源于
$$
P(A \cap B) = P(A \mid B) * P(B) = P(B \mid A) * P(A)
$$

2. **求和法则**：
$$
P(B) = \sum_{A} P(A \cap B)
$$

3. **期望**：
$$
E[X] = \sum_{x} x P(X = x).
$$
当$f(x)$的输入为从分布$P$中抽取的随机变量时，$f(x)$的期望值为：
$$
E_{x \sim P}[f(x)] = \sum_x f(x) P(x).
$$

4. 方差：
$$
\mathrm{Var}[X] = E\left[(X - E[X])^2\right] =
E[X^2] - E[X]^2.
$$

5. **马尔可夫链**：简要介绍参考[知乎专栏](https://zhuanlan.zhihu.com/p/448575579)

# 三、分布

In [ ]:
%matplotlib inline
from math import erf, factorial
import torch
from IPython import display
from d2l import torch as d2l

torch.pi = torch.acos(torch.zeros(1)) * 2  # Define pi in torch

## 3.1 伯努利分布

1. 可视化

In [ ]:
p = 0.3

d2l.set_figsize()
d2l.plt.stem([0, 1], [1 - p, p], use_line_collection=True)
d2l.plt.xlabel('x')
d2l.plt.ylabel('p.m.f.')
d2l.plt.show()

2. 代码实现：

In [ ]:
1*(torch.rand(10, 10) < p)

## 3.2 离散均匀分布

1. 可视化

In [ ]:
n = 5

d2l.plt.stem([i+1 for i in range(n)], n*[1 / n], use_line_collection=True)
d2l.plt.xlabel('x')
d2l.plt.ylabel('p.m.f.')
d2l.plt.show()

2. 代码实现

In [ ]:
torch.randint(1, n, size=(10, 10))

## 3.3 连续均匀分布

1. 可视化

In [ ]:
a, b = 1, 3

x = torch.arange(0, 4, 0.01)
p = (x > a).type(torch.float32)*(x < b).type(torch.float32)/(b-a)
d2l.plot(x, p, 'x', 'p.d.f.')

2. 代码实现

In [ ]:
(b - a) * torch.rand(10, 10) + a

## 3.4 二项分布

1. 可视化

In [ ]:
n, p = 10, 0.2

# Compute binomial coefficient
def binom(n, k):
    comb = 1
    for i in range(min(k, n - k)):
        comb = comb * (n - i) // (i + 1)
    return comb

pmf = torch.tensor([p**i * (1-p)**(n - i) * binom(n, i) for i in range(n + 1)])

d2l.plt.stem([i for i in range(n + 1)], pmf, use_line_collection=True)
d2l.plt.xlabel('x')
d2l.plt.ylabel('p.m.f.')
d2l.plt.show()

2. 代码实现

In [ ]:
m = torch.distributions.binomial.Binomial(n, p)
m.sample(sample_shape=(10, 10))

## 3.5 泊松分布

1. 可视化

In [ ]:
lam = 5.0

xs = [i for i in range(20)]
pmf = torch.tensor([torch.exp(torch.tensor(-lam)) * lam**k
                    / factorial(k) for k in xs])

d2l.plt.stem(xs, pmf, use_line_collection=True)
d2l.plt.xlabel('x')
d2l.plt.ylabel('p.m.f.')
d2l.plt.show()

2. 代码实现

In [ ]:
m = torch.distributions.poisson.Poisson(lam)
m.sample(sample_shape=(10, 10))

## 3.6 高斯分布(正态分布)

1. 可视化

In [ ]:
p = 0.2
ns = [1, 10, 100, 1000]
d2l.plt.figure(figsize=(10, 3))
for i in range(4):
    n = ns[i]
    pmf = torch.tensor([p**i * (1-p)**(n-i) * binom(n, i)
                        for i in range(n + 1)])
    d2l.plt.subplot(1, 4, i + 1)
    d2l.plt.stem([(i - n*p)/torch.sqrt(torch.tensor(n*p*(1 - p)))
                  for i in range(n + 1)], pmf,
                 use_line_collection=True)
    d2l.plt.xlim([-4, 4])
    d2l.plt.xlabel('x')
    d2l.plt.ylabel('p.m.f.')
    d2l.plt.title("n = {}".format(n))
d2l.plt.show()

mu, sigma = 0, 1

x = torch.arange(-3, 3, 0.01)
p = 1 / torch.sqrt(2 * torch.pi * sigma**2) * torch.exp(
    -(x - mu)**2 / (2 * sigma**2))

d2l.plot(x, p, 'x', 'p.d.f.')

2. 代码实现

In [ ]:
torch.normal(mu, sigma, size=(10, 10))